In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.2:3b",
    temperature = 0,
    base_url='http://localhost:11434'
)

Usando apenas o LLM

In [2]:
response = llm.invoke("Qual capitulo da lgpd trata do tratamento de dados pessoais pelo poder publico?")
print(response.content)


Olá!

A Lei Geral de Proteção de Dados (LGPD) é uma lei brasileira que regula a proteção de dados pessoais e a sua utilização por parte de entidades públicas.

O capítulo específico da LGPD que trata do tratamento de dados pessoais pelo poder público é o Capítulo VIII, que é composto pelos artigos 43 a 55.

Aqui estão alguns pontos importantes relacionados ao tratamento de dados pessoais pelo poder público, conforme estabelecido na LGPD:

*   Art. 43: O Poder Público deve garantir a proteção dos dados pessoais e a sua utilização apenas para fins previstos em lei.
*   Art. 44: A coleta, armazenamento e tratamento de dados pessoais pelo Poder Público devem ser realizados por meio de instrumentos legais ou regulamentares que garantam a proteção dos dados.
*   Art. 45: O Poder Público deve informar os cidadãos sobre as razões da coleta, armazenamento e tratamento de seus dados pessoais.
*   Art. 46: A coleta, armazenamento e tratamento de dados pessoais pelo Poder Público devem ser realiza

Obtemos a resposta errada

In [3]:
import pdfplumber
from pytesseract import image_to_string

def extract_text_with_ocr(pdf_path):
    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:

            if page.extract_text():
                extracted_text.append(page.extract_text())
            else:
                page_image = page.to_image(resolution=300)
                image = page_image.original  
                text = image_to_string(image)
                extracted_text.append(text)
    return extracted_text




extração dos dados do pdf

In [4]:
pdf_path = "data/lgpd.pdf"
text = extract_text_with_ocr(pdf_path)

print(text[:50])




['DIARIO OFICIAL DA UNIAO\n\nPublicado em: 15/08/2018 | Edic&o: 157 | Segao: 1 | Pagina: 59\nOrgao: Atos do Poder Legislativo\n\nLEI NO 13.709, DE 14 DE AGOSTO DE 2018\n\nDisp6e sobre a protegao de dados pessoais e altera a Lei n°\n12.965, de 23 de abril de 2014 (Marco Civil da Internet).\n\nOPRESIDENTEDAREPUBLICA\nFago saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:\nCAPITULO |\nDISPOSICOES PRELIMINARES\n\nArt. 1° Esta Lei dispoe sobre o tratamento de dados pessoais, inclusive nos meios digitais, por\npessoa natural ou por pessoa juridica de direito publico ou privado, com o objetivo de proteger os direitos\nfundamentais de liberdade e de privacidade e o livre desenvolvimento da personalidade da pessoa\nnatural.\n\nArt. 2° A disciplina da protecao de dados pessoais tem como fundamentos:\n| - o respeito a privacidade:\n\nIl - a autodeterminacao informativa;\n\nIll - a liberdade de expressao, de informacao, de comunicagao e de opiniao;\nIV - a inviolabilidade da int

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document, HumanMessage
from langchain.vectorstores import FAISS



def create_documents(texts):
    if isinstance(texts, list):
        documents = [Document(page_content=text) for text in texts]
    else:
        documents = [Document(page_content=texts)]
    return documents

def create_faiss_vectorstore(documents):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

def generate_response_with_ollama(vectorstore, query):
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(query)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    
    print("Contexto Recuperado:")
    print(context[:200])
    
    prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()

def rag_pipeline_with_ollama(texts, query):
    documents = create_documents(texts)
    vectorstore = create_faiss_vectorstore(documents)
    response = generate_response_with_ollama(vectorstore, query)
    return response




In [6]:
query = "Qual capítulo da LGPD trata do tratamento de dados pessoais pelo poder público?"
response = rag_pipeline_with_ollama(text, query)

# Exibir resposta gerada
print("\nResposta Gerada:", response)

/home/airex/anaconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Contexto Recuperado:
DIARIO OFICIAL DA UNIAO

Publicado em: 15/08/2018 | Edic&o: 157 | Segao: 1 | Pagina: 59
Orgao: Atos do Poder Legislativo

LEI NO 13.709, DE 14 DE AGOSTO DE 2018

Disp6e sobre a protegao de dados pesso

Resposta Gerada: O capítulo que trata do tratamento de dados pessoais pelo poder público é o Capítulo IV, "Tratamento de Dados Pessoais por Poder Público".


multiplos_documentos

In [7]:
import os
import pdfplumber
from pytesseract import image_to_string
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_ollama import ChatOllama


In [8]:
def extract_text_with_ocr(pdf_path):

    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                extracted_text.append(page.extract_text())
            else:
                page_image = page.to_image(resolution=300)
                image = page_image.original
                text = image_to_string(image, lang="por")
                extracted_text.append(text)
    return "\n".join(extracted_text)


In [9]:
def load_pdf_documents(pdf_paths):

    all_documents = []
    for path in pdf_paths:
        print(f"Carregando: {path}")
        try:
            # Extract and consolidate text from the PDF
            text = extract_text_with_ocr(path)
            filename = os.path.basename(path)
            document = Document(
                page_content=text,
                metadata={"source": filename}  # Metadata includes the file name
            )
            all_documents.append(document)
            print(f"Documento consolidado para {filename}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {e}")

    if not all_documents:
        raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
    print(f"Total de documentos carregados: {len(all_documents)}")
    return all_documents


In [10]:
def create_vectorstore(documents):

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore


In [11]:
def generate_response(query, vectorstore):

    docs = vectorstore.similarity_search(query, k=5)
    context = "\n\n".join([
        f"Source: {doc.metadata.get('source', 'Unknown')}\nContent: {doc.page_content}" for doc in docs
    ])
    
    llm = ChatOllama(
        model="llama3.2:3b",
        temperature=0,
        base_url="http://localhost:11434"
    )

    prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
    response = llm.invoke([prompt])
    
    output = {
        "question": query,
        "context": context[:500],
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Unknown") for doc in docs]
    }
    return output


In [12]:
def prepare_vectorstore(pdf_paths):

    documents = load_pdf_documents(pdf_paths)
    vectorstore = create_vectorstore(documents)
    print("Vectorstore criado com sucesso.")
    return vectorstore


In [13]:
def ask_question(query, vectorstore):

    response = generate_response(query, vectorstore)
    return response


In [14]:
pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore = prepare_vectorstore(pdf_files)

Carregando: data/lgpd.pdf
Documento consolidado para lgpd.pdf
Carregando: data/codigo_civil.pdf
Documento consolidado para codigo_civil.pdf
Total de documentos carregados: 2


/tmp/ipykernel_2254/2212122749.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Vectorstore criado com sucesso.


In [15]:
# Fazer uma pergunta
query1 = "Quais são os princípios da LGPD?"
response1 = ask_question(query1, vectorstore)

# Exibir a resposta
print("\nResposta Gerada:")
print(response1["response"])
print("Pergunta:", response1["question"])
print("Contexto:", response1["context"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Fazer outra pergunta sem recarregar os documentos
query2 = "Qual capitulo trata da sociedade anonima no código civil?"
response2 = ask_question(query2, vectorstore)

# Exibir a nova resposta
print("\nResposta Gerada:")
print(response2["response"])
print("Contexto:", response2["context"])

print("Pergunta:", response2["question"])

print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)



Resposta Gerada:
Os princípios da Lei Geral de Proteção de Dados (LGPD) incluem a proteção dos dados pessoais, a transparência e a responsabilidade, a consentimento informado, a segurança e a confidencialidade, a privacidade e a proteção contra o uso indevido.
Pergunta: Quais são os princípios da LGPD?
Contexto: Source: lgpd.pdf
Content: DIÁRIO OFICIAL DA UNIÃO

Publicado em: 15/08/2018 | Edição: 157 | Seção: 1 | Página: 59
Órgão: Atos do Poder Legislativo

LEI NO 13.709, DE 14 DE AGOSTO DE 2018

Dispõe sobre a proteção de dados pessoais e altera a Lei nº
12.965, de 23 de abril de 2014 (Marco Civil da Internet).

OPRESIDENTEDAREPÚBLICA
Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:
CAPÍTULO |
DISPOSIÇÕES PRELIMINARES

Art. 1º Esta Lei dispõe sobre o tratamento de dados pessoai

Fontes:
- lgpd.pdf
- codigo_civil.pdf

Resposta Gerada:
O Capítulo III do Código Civil Brasileiro trata da Sociedade Anônima.
Contexto: Source: lgpd.pdf
Content: DIÁRIO OFICIAL DA UNI

usar chunks

In [16]:
def split_text_into_chunks(text, chunk_size=1000, overlap=200):

    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap if end - overlap > start else end
    return chunks


In [17]:
def load_pdf_documents_with_chunks(pdf_paths, chunk_size=1000, overlap=200):

    all_documents = []
    for path in pdf_paths:
        print(f"Carregando: {path}")
        try:
            text = extract_text_with_ocr(path)
            filename = os.path.basename(path)
            chunks = split_text_into_chunks(text, chunk_size, overlap)
            for i, chunk in enumerate(chunks):
                document = Document(
                    page_content=chunk,
                    metadata={"source": filename, "chunk_index": i}
                )
                all_documents.append(document)
            print(f"{len(chunks)} chunks criados para {filename}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {e}")

    if not all_documents:
        raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
    print(f"Total de chunks carregados: {len(all_documents)}")
    return all_documents


In [18]:
def prepare_vectorstore_with_chunks(pdf_paths, chunk_size=1000, overlap=200):

    documents = load_pdf_documents_with_chunks(pdf_paths, chunk_size, overlap)
    vectorstore = create_vectorstore(documents)
    print("Vectorstore criado com chunks.")
    return vectorstore


In [19]:
# Carregar documentos, dividir em chunks e criar vetorstore
pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore_with_chunks = prepare_vectorstore_with_chunks(pdf_files, chunk_size=1000, overlap=200)


Carregando: data/lgpd.pdf
72 chunks criados para lgpd.pdf
Carregando: data/codigo_civil.pdf
1666 chunks criados para codigo_civil.pdf
Total de chunks carregados: 1738
Vectorstore criado com chunks.


In [20]:
# Fazer uma pergunta
query1 = "definicao lei geral de protecao de dados"
response1 = ask_question(query1, vectorstore_with_chunks)

# Exibir a resposta
print("\nResposta Gerada:")
print(response1["response"])
print("Pergunta:", response1["question"])


print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Fazer outra pergunta sem recarregar os documentos
query2 = "capitulo das perdas e danos?"
response2 = ask_question(query2, vectorstore_with_chunks)

# Exibir a nova resposta
print("\nResposta Gerada:")
print(response2["response"])


print("Pergunta:", response2["question"])

print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)



Resposta Gerada:
A Lei Geral de Proteção de Dados Pessoais (LGPD) define a proteção de dados pessoais como um direito fundamental que visa garantir a privacidade e a segurança dos indivíduos em relação à coleta, armazenamento e tratamento de suas informações pessoais. A LGPD estabelece regras e princípios para proteger os dados pessoais, incluindo:

1. Princípio da transparência: é obrigatório informar o titular dos dados sobre a coleta, uso e compartilhamento de seus dados.
2. Princípio da consentimento: é necessário obter o consentimento do titular para coletar, armazenar e tratar seus dados pessoais.
3. Princípio da segurança: os dados pessoais devem ser protegidos contra acessos não autorizados, perda, dano ou destruição.
4. Princípio da confidencialidade: os dados pessoais devem ser mantidos confidenciais e não compartilhados sem consentimento do titular.
5. Princípio da responsabilidade: os controladores e operadores de dados são responsáveis por garantir a proteção dos dados pe

In [21]:
def summarize_history_with_previous(history):
    """
    Inclua todo o histórico, destacando explicitamente a interação mais recente.
    """
    if not history:
        return "Histórico vazio."

    # Última interação (pergunta e resposta anterior)
    last_interaction = f"Pergunta Anterior: {history[-1][0]}\nResposta Anterior: {history[-1][1]}"
    
    # Outras interações (exceto a última)
    previous_interactions = "\n".join([
        f"Pergunta: {q}\nResposta: {r}" for q, r in history[:-1]
    ])
    
    # Combinar tudo
    return f"{previous_interactions}\n\n{last_interaction}" if previous_interactions else last_interaction


def format_prompt_with_history(query, context, history):
    """
    Gera o prompt incluindo o histórico completo de interações e o contexto.
    Integra explicitamente a última pergunta e resposta no contexto.
    """
    # Incluir o histórico completo
    full_history = summarize_history_with_previous(history)
    
    # Destacar a última interação diretamente no contexto
    if history:
        last_question, last_answer = history[-1]
        last_interaction_context = f"Baseando-se na última interação:\nPergunta: {last_question}\nResposta: {last_answer}\n"
    else:
        last_interaction_context = ""

    # Construir o prompt
    prompt = f"""
    Você é um assistente jurídico especializado no Código Civil brasileiro.

    Última Interação:
    {last_interaction_context}

    Histórico de Conversa:
    {full_history}

    Trechos relevantes dos documentos:
    {context}

    Nova Pergunta:
    {query}

    Resposta:
    """
    return prompt


def generate_response_with_history(query, vectorstore, history):
    """
    Gera resposta considerando o histórico completo, com destaque para a pergunta anterior, e o contexto relevante.
    """
    # Buscar os documentos mais relevantes no Vectorstore
    docs = vectorstore.similarity_search(query, k=5)
    
    # Limitar o contexto ao mais relevante
    context = "\n\n".join([
        f"Fonte: {doc.metadata.get('source', 'Desconhecido')}\nTrecho: {doc.page_content[:300]}" for doc in docs
    ])
    
    # Configurar o LLM
    llm = ChatOllama(
        model="llama3.2:3b",
        temperature=0,
        base_url="http://localhost:11434"
    )
    
    # Gerar o prompt com histórico e contexto
    prompt = format_prompt_with_history(query, context, history)
    
    # Obter a resposta do modelo
    response = llm.invoke([prompt])
    
    # Adicionar a interação ao histórico ANTES de retornar
    history.append((query, response.content.strip()))
    
    # Formatar e retornar o resultado
    output = {
        "question": query,
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Desconhecido") for doc in docs],
        "context_used": context  # Adicionado para debugging
    }
    return output


def ask_question_with_history(query, vectorstore, history):
    """
    Wrapper para gerar respostas com histórico.
    """
    return generate_response_with_history(query, vectorstore, history)


# Exemplo de Uso com Histórico Corrigido
conversation_history = []

# Pergunta 1
response1 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response1["response"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Pergunta 2 (Nova pergunta com base na resposta anterior)
response2 = ask_question_with_history(
    '''Os capítulos do Livro II do Código Civil são Livro II – Dos Bens
Título Único – Das Diferentes Classes de Bens

Capítulo I – Dos Bens Considerados em Si Mesmos
Seção I – Dos Bens Imóveis
Seção II – Dos Bens Móveis
Seção III – Dos Bens Fungíveis e Consumíveis
Seção IV – Dos Bens Divisíveis
Seção V – Dos Bens Singulares e Coletivos
Capítulo II – Dos Bens Reciprocamente Considerados
Capítulo III – Dos Bens Públicos''', 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response2["response"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)

# Pergunta 3 (Outra pergunta baseada no fluxo de conversa)
response3 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response3["response"])
print("\nFontes:")
for source in set(response3["sources"]):
    print("-", source)



Resposta Gerada:
Olá! Estou aqui para ajudar.

O Livro II do Código Civil brasileiro é dividido em várias seções, que abordam diferentes aspectos do direito privado. Aqui estão os capítulos e seções principais:

**Capítulo I: Do Patrimônio**

* Seção 1ª: Da propriedade
* Seção 2ª: Da herança

**Capítulo II: Das Pessoas Jurídicas de Direito Privado**

* Seção 1ª: Da sociedade
* Seção 2ª: Do testamento
* Seção 3ª: Da sucessão
* Seção 4ª: Da doação e da herança

**Capítulo III: Das Obligações**

* Seção 1ª: Da obrigação de fazer ou não fazer
* Seção 2ª: Da obrigação de pagar
* Seção 3ª: Da obrigação de entrega
* Seção 4ª: Da obrigação de não fazer

**Capítulo IV: Das Contratos**

* Seção 1ª: Do contrato em geral
* Seção 2ª: Do contrato de compra e venda
* Seção 3ª: Do contrato de locação
* Seção 4ª: Do contrato de trabalho

Essas são as seções principais do Livro II do Código Civil. É importante notar que cada seção tem seus próprios capítulos e artigos, que detalham as regras e normas e

melhora os chunks

In [32]:
# import re
# def split_chunks_v2(text, chunk_size=1000, overlap=200):

#     # Padrões para detectar Livro, Capítulo e Seção (case-insensitive)
#     livro_pattern = re.compile(r"(Livro\s+[IVXLCDM\d]+\s*.*?)\s*(?=\n|$)", re.IGNORECASE)
#     capitulo_pattern = re.compile(r"(Capítulo\s+[IVXLCDM\d]+\s*.*?)\s*(?=\n|$)", re.IGNORECASE)
#     secao_pattern = re.compile(r"(Seção\s+[IVXLCDM\d]+\s*.*?)\s*(?=\n|$)", re.IGNORECASE)


#     # Variáveis para acompanhar os metadados atuais
#     current_livro = None
#     current_capitulo = None
#     current_secao = None

#     # Lista de chunks e variáveis de controle
#     chunks = []
#     current_chunk = []
#     current_size = 0

#     def save_chunk_and_reset():

#         nonlocal current_chunk, current_size
#         if current_chunk:
#             chunks.append({
#                 "content": "\n".join(current_chunk),
#                 "metadata": {
#                     "livro": current_livro,
#                     "capítulo": current_capitulo,
#                     "seção": current_secao
#                 }
#             })
#             # Reiniciar mantendo o overlap
#             current_chunk = current_chunk[-overlap:]
#             current_size = sum(len(line) for line in current_chunk)

#     # Processar linhas do texto
#     for line in text.split("\n"):
#         # Detectar mudanças nos metadados
#         if livro_match := livro_pattern.match(line):
#             save_chunk_and_reset()
#             current_livro = livro_match.group(1)
#             # print(f"Livro detectado: {current_livro}")

#         if capitulo_match := capitulo_pattern.match(line):
#             save_chunk_and_reset()
#             current_capitulo = capitulo_match.group(1)
#             # print(f"Capítulo detectado: {current_capitulo}")

#         if secao_match := secao_pattern.match(line):
#             save_chunk_and_reset()
#             current_secao = secao_match.group(1)
#             # print(f"Seção detectada: {current_secao}")

#         # Adicionar linha ao chunk atual
#         current_chunk.append(line)
#         current_size += len(line)

#         # Criar chunk se atingir o tamanho definido
#         if current_size >= chunk_size:
#             save_chunk_and_reset()

#     # Adicionar o último chunk
#     save_chunk_and_reset()


#     return chunks


import re

def split_chunks_v2(text, chunk_size=1000, overlap=200):

    # Padrões para detectar Livro, Capítulo e Seção
    livro_pattern = re.compile(r"(Livro\s+[IVXLCDM\d]+.*?)\s*(?=\n|$)", re.IGNORECASE)
    capitulo_pattern = re.compile(r"(Capítulo\s+[IVXLCDM\d]+.*?)\s*(?=\n|$)", re.IGNORECASE)
    secao_pattern = re.compile(r"(Seção\s+[IVXLCDM\d]+.*?)\s*(?=\n|$)", re.IGNORECASE)

    # Variáveis para acompanhar os metadados atuais
    current_livro = None
    current_capitulo = None
    current_secao = None

    # Lista de chunks e variáveis de controle
    chunks = []
    current_chunk = []
    current_size = 0
    chunk_index = 0  # Indexação dos chunks

    def save_chunk_and_reset():
        """Salva o chunk atual, adiciona o índice e reinicia mantendo a sobreposição."""
        nonlocal current_chunk, current_size, chunk_index
        if current_chunk:
            chunks.append({
                "chunk_index": chunk_index,
                "content": "\n".join(current_chunk),
                "metadata": {
                    "livro": current_livro,
                    "capítulo": current_capitulo,
                    "seção": current_secao
                }
            })
            chunk_index += 1
            # Reiniciar mantendo o overlap
            current_chunk = current_chunk[-overlap:]
            current_size = sum(len(line) for line in current_chunk)

    # Processar linhas do texto
    for line in text.split("\n"):
        # Detectar mudanças nos metadados
        if livro_match := livro_pattern.match(line):
            save_chunk_and_reset()
            current_livro = livro_match.group(1)

        if capitulo_match := capitulo_pattern.match(line):
            save_chunk_and_reset()
            current_capitulo = capitulo_match.group(1)

        if secao_match := secao_pattern.match(line):
            save_chunk_and_reset()
            current_secao = secao_match.group(1)

        # Adicionar linha ao chunk atual
        current_chunk.append(line)
        current_size += len(line)

        # Criar chunk se atingir o tamanho definido
        if current_size >= chunk_size:
            save_chunk_and_reset()

    # Adicionar o último chunk
    save_chunk_and_reset()

    return chunks


In [30]:
def load_pdf_documents_v2(pdf_paths, chunk_size=1000, overlap=200):

    documents = []
    for pdf_path in pdf_paths:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        chunks = split_chunks_v2(text, chunk_size, overlap)
                        for chunk in chunks:
                            documents.append(Document(
                                page_content=chunk["content"],
                                metadata={
                                    "source": pdf_path,
                                    **chunk["metadata"]  # Adiciona Livro, Capítulo, Parágrafo
                                }
                            ))
        except Exception as e:
            print(f"Erro ao processar {pdf_path}: {e}")
    return documents


In [24]:
def prepare_vectorstore_with_chunks_v2(pdf_paths, chunk_size=1000, overlap=200):
    from langchain.vectorstores import FAISS
    from langchain.embeddings import HuggingFaceEmbeddings

    # Carregar documentos estruturados
    documents = load_pdf_documents_v2(pdf_paths, chunk_size, overlap)

    # Criar vetorstore
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    
    return vectorstore


In [33]:

pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore_with_chunks_v2 = prepare_vectorstore_with_chunks_v2(pdf_files, chunk_size=1000, overlap=200)




Livro detectado: Livro I – Das Pessoas
Capítulo detectado: Capítulo I – Da Personalidade e da Capacidade .............................................143
Capítulo detectado: Capítulo II – Dos Direitos da Personalidade ..................................................145
Capítulo detectado: Capítulo III – Da Ausência
Seção detectada: Seção I – Da Curadoria dos Bens do Ausente .............................................146
Seção detectada: Seção II – Da Sucessão Provisória ..............................................................146
Seção detectada: Seção III – Da Sucessão Definitiva .............................................................148
Capítulo detectado: Capítulo I – Disposições Gerais .....................................................................148
Capítulo detectado: Capítulo II – Das Associações ........................................................................151
Capítulo detectado: Capítulo III – Das Fundações ......................................

In [ ]:
from sentence_transformers import CrossEncoder

def rerank_chunks(query, docs, re_ranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2"):
    """
    Reordena os chunks recuperados com base na relevância usando um modelo de re-ranking.
    """
    re_ranker = CrossEncoder(re_ranker_model)
    pairs = [(query, doc.page_content) for doc in docs]
    scores = re_ranker.predict(pairs)
    ranked_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked_docs]


In [35]:
from sentence_transformers import CrossEncoder

# Função de re-ranking
def rerank_chunks(query, docs, re_ranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2"):
    """
    Reclassifica os chunks com base na relevância em relação à query usando um modelo de re-ranking.
    """
    re_ranker = CrossEncoder(re_ranker_model)
    pairs = [(query, doc.page_content) for doc in docs]
    scores = re_ranker.predict(pairs)
    ranked_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked_docs]

# Formatação do prompt com histórico
def format_prompt_with_history(query, context, history):
    """
    Formata o prompt incluindo o histórico da conversa e o contexto relevante.
    """
    full_history = "\n".join([f"Pergunta: {q}\nResposta: {r}" for q, r in history])
    return f"""
    Histórico da Conversa:
    {full_history}

    Contexto relevante:
    {context}

    Pergunta:
    {query}

    Responda com base no contexto acima. Caso a resposta não esteja clara, indique que a informação não foi encontrada.
    """

# Gerar resposta com histórico e re-ranking
def ask_question_with_history(query, vectorstore, history, k=5, re_ranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2"):
    """
    Recupera chunks, aplica re-ranking e gera uma resposta considerando o histórico.
    """
    # Recupera os chunks iniciais
    docs = vectorstore.similarity_search(query, k=k)

    # Aplica re-ranking para priorizar chunks mais relevantes
    ranked_docs = rerank_chunks(query, docs, re_ranker_model)

    # Formata o contexto com os chunks reclassificados
    context = "\n\n".join([f"Fonte: {doc.metadata.get('source', 'Desconhecido')}\n{doc.page_content}" for doc in ranked_docs])

    # Formata o prompt incluindo o histórico
    prompt = format_prompt_with_history(query, context, history)

    # Gera a resposta usando o LLM
    response = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    # Atualiza o histórico
    history.append((query, response))

    return {
        "response": response,
        "sources": [doc.metadata.get("source", "Desconhecido") for doc in ranked_docs],
        "context_used": context
    }


In [36]:
conversation_history = []

# Pergunta 1
response1 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?",
    vectorstore_with_chunks_v2,
    conversation_history
)

print("\nResposta Gerada:")
print(response1["response"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Pergunta 2
response2 = ask_question_with_history(
    """Os capítulos do Livro II do Código Civil são: Livro II – Dos Bens
Título Único – Das Diferentes Classes de Bens

Capítulo I – Dos Bens Considerados em Si Mesmos
Seção I – Dos Bens Imóveis
Seção II – Dos Bens Móveis
Seção III – Dos Bens Fungíveis e Consumíveis
Seção IV – Dos Bens Divisíveis
Seção V – Dos Bens Singulares e Coletivos
Capítulo II – Dos Bens Reciprocamente Considerados
Capítulo III – Dos Bens Públicos""",
    vectorstore_with_chunks_v2,
    conversation_history
)

print("\nResposta Gerada:")
print(response2["response"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)

# Pergunta 3
response3 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?",
    vectorstore_with_chunks_v2,
    conversation_history
)

print("\nResposta Gerada:")
print(response3["response"])
print("\nFontes:")
for source in set(response3["sources"]):
    print("-", source)



Resposta Gerada:
Infelizmente, não encontrei informações sobre os capítulos e seções do Livro II do Código Civil no documento fornecido. O documento parece ser uma fonte de informação geral sobre o Código Civil brasileiro, mas não contém detalhes específicos sobre a estrutura do Livro II.

Se você puder fornecer mais contexto ou informações sobre o Livro II do Código Civil, posso tentar ajudá-lo a encontrar as respostas que está procurando.

Fontes:
- data/codigo_civil.pdf

Resposta Gerada:
Com base no contexto fornecido, parece que o Livro II do Código Civil é dividido em capítulos e seções como segue:

Livro II - Dos Bens
Título Único - Das Diferentes Classes de Bens

Capítulo I - Dos Bens Considerados em Si Mesmos
Seção I - Dos Bens Imóveis
Seção II - Dos Bens Móveis
Seção III - Dos Bens Fungíveis e Consumíveis
Seção IV - Dos Bens Divisíveis
Seção V - Dos Bens Singulares e Coletivos

Capítulo II - Dos Bens Reciprocamente Considerados

Capítulo III - Dos Bens Públicos

Fontes:
- dat

In [34]:



conversation_history = []
# Pergunta 1

response1 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks_v2, 
    conversation_history
)

print("\nResposta Gerada:")
print(response1["response"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Pergunta 2 (Nova pergunta com base na resposta anterior)
response2 = ask_question_with_history(
    '''Os capítulos do Livro II do Código Civil são: Livro II – Dos Bens
Título Único – Das Diferentes Classes de Bens

Capítulo I – Dos Bens Considerados em Si Mesmos
Seção I – Dos Bens Imóveis
Seção II – Dos Bens Móveis
Seção III – Dos Bens Fungíveis e Consumíveis
Seção IV – Dos Bens Divisíveis
Seção V – Dos Bens Singulares e Coletivos
Capítulo II – Dos Bens Reciprocamente Considerados
Capítulo III – Dos Bens Públicos''', 
    vectorstore_with_chunks_v2, 
    conversation_history
)

print("\nResposta Gerada:")
print(response2["response"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)

# Pergunta 3 (Outra pergunta baseada no fluxo de conversa)
response3 = ask_question_with_history(
    "Quais são os capítulos e secoes do Livro II do Código Civil?", 
    vectorstore_with_chunks_v2, 
    conversation_history
)

print("\nResposta Gerada:")
print(response3["response"])
print("\nFontes:")
for source in set(response3["sources"]):
    print("-", source)



Resposta Gerada:
Olá! Estou aqui para ajudar.

O Livro II do Código Civil brasileiro é conhecido como "Das Pessoas". Ele é dividido em 5 capítulos e 34 seções. Aqui está a estrutura detalhada:

**Capítulo I: Da Personalidade**

* Seção I: Da personalidade jurídica
* Seção II: Do estado de nulidade da personalidade
* Seção III: Da personalidade jurídica das pessoas físicas e jurídicas

**Capítulo II: Da Capacidade para Contratar**

* Seção I: Da capacidade geral para contratar
* Seção II: Da incapacidade para contratar
* Seção III: Das condições da capacidade para contratar

**Capítulo III: Da Propriedade**

* Seção I: Do direito de propriedade
* Seção II: Dos direitos relativos à propriedade
* Seção III: Da alienação do imóvel

**Capítulo IV: Das Obrações e dos Negócios Justos**

* Seção I: Da obração
* Seção II: Do negócio justo
* Seção III: Dos negócios justos que não se enquadram na categoria anterior

**Capítulo V: Da Família**

* Seção I: Da família
* Seção II: Das relações de pa

usar agent

In [28]:
from langchain.agents import Tool
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA



retrieval_qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore_with_chunks.as_retriever())

tools = [
    Tool(
        name="Document Search",
        func=retrieval_qa.run,
        description=(

            "Use para buscar informações relevantes nos documentos. Perguntas devem ser específicas."
    
        )
    )
]



In [29]:
from langchain.agents import initialize_agent, AgentType

# Inicializar o Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True ,
    handle_parsing_errors=True,
    max_iterations=1

)


/tmp/ipykernel_21970/1202134613.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [30]:
# Consultar o Agent
query1 = "definicao lei geral de protecao de dados"
query2 = "capitulo das perdas e danos?"

response1 = agent.run(query1)
response2 = agent.run(query2)

# Exibir as respostas
print("Resposta 1:", response1)
print("Resposta 2:", response2)


/tmp/ipykernel_21970/3719747913.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run(query1)




> Entering new AgentExecutor chain...
Question: definicao lei geral de protecao de dados
Thought: Preciso encontrar a definição da Lei Geral de Proteção de Dados (LGPD) no Brasil.
Action: Document Search
Action Input: "Lei Geral de Proteção de Dados"
Observation: A Lei Geral de Proteção de Dados (LGPD) é uma lei brasileira que regula a proteção de dados pessoais em todo o território nacional. Ela foi publicada no Diário Oficial da União em 10 de outubro de 2018 e entrou em vigor em 15 de setembro de 2020.

A LGPD tem como objetivo proteger os direitos dos cidadãos brasileiros em relação à sua vida pessoal, a privacidade e a segurança dos dados pessoais. Ela estabelece regras para a coleta, armazenamento, tratamento e compartilhamento de dados pessoais, bem como para a proteção desses dados contra violações.

Algumas das principais disposições da LGPD incluem:

* A necessidade de obter o consentimento informado do titular dos dados antes de coletar, armazenar ou tratá-los;
* A obrigaç